###  What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

####  Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
# chain creation
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser 
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
import os
load_dotenv()

groq_llm = ChatGroq(model="llama-3.1-8b-instant",api_key=os.getenv("GROQ_API"),temperature=0.2)
loader = TextLoader("G:\\extracted\\ML\\RAG\\data_management\\text_files\\حباب سکه چیست.txt",encoding="utf-8")

loader = loader.load()

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=600,
    chunk_overlap=50 # specially make it more to increase relevancy 
)
docs = splitter.split_documents(loader)

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

vectorstore = FAISS.from_documents(docs, embedding_model)

retriever = vectorstore.as_retriever(
    search_type="mmr", # ('similarity', 'similarity_score_threshold', 'mmr')
    search_kwargs={"k":3, "lambda_mult": 0.7})

In [40]:
# query enhancement prompt
decomposition_prompt = PromptTemplate.from_template("""
سوال پیچیده زیر را برای بازیابی بهتر اسناد به ۲ تا ۴ زیر سوال کوچکتر تجزیه کنید.
در مجموع 4 زیر سوال بنویس

سوال دریافتی: {query}

""")
decomposition_chain = decomposition_prompt | groq_llm | StrOutputParser()

In [41]:
print(decomposition_chain.invoke({'query':'حباب منفی چیست و درباره سکه چطور است و خوبه یا نه'}))

سوال پیچیده را به ۴ زیر سوال کوچکتر تجزیه می‌کنیم:

1. **حباب منفی چیست؟** : حباب منفی چیست و چه ویژگی‌هایی دارد؟ آیا این حباب‌ها در سکه‌ها وجود دارند یا خیر؟
2. **حباب منفی در سکه‌ها چه نقش‌ای دارد؟** : آیا حباب منفی در سکه‌ها وجود دارد و چه تاثیری بر ارزش سکه دارد؟
3. **حباب منفی خوبه یا نه؟** : آیا حباب منفی در سکه‌ها خوب است یا بد؟ چه عواملی بر ارزش حباب منفی در سکه‌ها تأثیر می‌گذارد؟
4. **چگونه حباب منفی در سکه‌ها را تشخیص دهیم؟** : چگونه می‌توانیم حباب منفی در سکه‌ها را تشخیص دهیم و از آن‌ها مطمئن باشیم؟


In [16]:
qa_prompt = PromptTemplate.from_template(
"""
تو یک دستیار فارسی‌زبان مفید هستی.
از اطلاعات داده‌شده برای پاسخ دقیق به سوال استفاده کن.
پاسخ باید روان، طبیعی، و مناسب خواندن توسط انسان باشد.
از استفاده از هر زبان یا کاراکتر غیر فارسی (مانند چینی، انگلیسی یا عربی) خودداری کن.


اطلاعات: {context}
سوال کاربر : {question}

"""
)
qa_chain = create_stuff_documents_chain(groq_llm,qa_prompt)

In [33]:
def extract_subquestions(text: str) -> list[str]:
    """
    Extract only real sub-questions from decomposition output.
    Supports:
    - Bold questions (**سوال؟**)
    - Normal Persian questions ending with ؟
    """

    questions = []

    for line in text.splitlines():
        line = line.strip()

        if not line:
            continue

        # Case 1: Bold question
        bold_match = re.match(r"\*\*(.+?\؟)\*\*", line)
        if bold_match:
            questions.append(bold_match.group(1))
            continue

        # Case 2: Normal question line
        if line.endswith("؟"):
            questions.append(line)

    return questions

In [48]:
import re
def full_query_decomposition(query):
    response = decomposition_chain.invoke({'query': query})
    sub_questions = re.findall(r'^\s*(?:\d+\.|-)\s*(.+)$', response, flags=re.MULTILINE) # not fine
    # sub_questions = [q.strip("-•1234567890. ").strip() for q in response.split("\n") if q.strip()]  # not fine
    # sub_questions = extract_subquestions(response)
    print(sub_questions)
    combined_answer =[]
    for i in sub_questions:
        retrieved = retriever.invoke(i)
        result = qa_chain.invoke({'context':retrieved,'question':i})
        combined_answer.append(f"Q: {i}\nA: {result}")
    return "\n\n".join(combined_answer)

In [49]:
query = "حباب منفی چیست و درباره سکه چطور است و خوبه یا نه"
final_answer = full_query_decomposition(query)
print("final answer:\n")
print(final_answer)

['**حباب منفی چیست؟** : حباب منفی چیست و چه ویژگی\u200cهایی دارد؟ آیا این حباب\u200cها در سکه\u200cها وجود دارند یا خیر؟', '**حباب منفی در سکه\u200cها چه نقش\u200cای دارد؟** : آیا حباب منفی در سکه\u200cها به عنوان یک ویژگی مثبت یا منفی در نظر گرفته می\u200cشود؟ آیا این حباب\u200cها به ارزش سکه\u200cها اضافه می\u200cکنند یا کم؟', '**آیا حباب منفی در سکه\u200cها وجود دارد؟** : آیا حباب منفی در سکه\u200cهای مختلف وجود دارد یا خیر؟ آیا این حباب\u200cها در سکه\u200cهای قدیمی\u200cتر یا جدیدتر دیده می\u200cشوند؟', '**حباب منفی خوبه یا نه؟** : آیا حباب منفی در سکه\u200cها به عنوان یک ویژگی مثبت در نظر گرفته می\u200cشود یا خیر؟ آیا این حباب\u200cها به ارزش سکه\u200cها اضافه می\u200cکنند یا کم؟']
final answer:

Q: **حباب منفی چیست؟** : حباب منفی چیست و چه ویژگی‌هایی دارد؟ آیا این حباب‌ها در سکه‌ها وجود دارند یا خیر؟
A: حباب منفی در بازار سکه، زمانی رخ می‌دهد که قیمت روز سکه از ارزش ذاتی طلای آن کمتر باشد. در این شرایط، قیمت سکه در بازار از قیمت واقعی آن کمتر است و این تفاوت قیمت را می‌توانیم حب